In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
#从文件中读取数据
alpha_df_17=pd.read_csv("factor_data.csv",dtype={"stock_code":str})
alpha_df_16=pd.read_csv("factor_data_16.csv",dtype={"stock_code":str})
alpha_df_15=pd.read_csv("factor_data_15.csv",dtype={"stock_code":str})
alpha_df_14=pd.read_csv("factor_data_14.csv",dtype={"stock_code":str})
alpha_df_13=pd.read_csv("factor_data_13.csv",dtype={"stock_code":str})
alpha_df_12=pd.read_csv("factor_data_12.csv",dtype={"stock_code":str})
alpha_df=pd.concat([alpha_df_17,alpha_df_16,alpha_df_15,alpha_df_14,alpha_df_13,alpha_df_12])

alpha_df.rename(columns={"date":"datetime","stock_code":"code"},inplace=True)
alpha_df["code"]=alpha_df["code"].apply(lambda x:x+".SH" if x[0]=="6" else x+".SZ")
alpha_df.sort_values(by=["datetime","code"],ascending=[False,True],inplace=True)

#添加上股票的行业信息
data=pd.read_csv("data_needs.csv")
date=list(data["datetime"].drop_duplicates())[-1]
df=data.groupby("datetime").get_group(date)
dic=dict(zip(df["code"],df["industry_code"]))
alpha_df["industry_code"]=alpha_df["code"].apply(lambda x:dic.get(x,0))

In [ ]:
alpha_df.set_index("code",inplace=True)

#去除st股票
st_stock=list(pd.read_csv("st_stocks.csv")["code"])
alpha_df.drop(st_stock,inplace=True)

#去除新股和停牌股
grouped=alpha_df.groupby("datetime")
date_list=list(alpha_df["datetime"].drop_duplicates())
code_list=list(alpha_df.index.drop_duplicates())
join=set(code_list)
union=set(code_list)
for date in date_list:
    df=grouped.get_group(date)
    stock_set=set(df.index)
    join=stock_set&join
    union=stock_set|union
other_list=list(union-join)
alpha_df.drop(other_list,inplace=True)

alpha_df.to_csv("zz_data.csv",index_label="code")

In [ ]:
#行业中位数填充缺失值
def median_fill(alpha_df,factor_list):
    grouped = alpha_df.groupby("datetime")
    date_list = list(alpha_df["datetime"].drop_duplicates())
    filled_df = pd.DataFrame()

    flag = 0

    for date in date_list:
        print(date,end=" ")
        panel_data = grouped.get_group(date)

        df = pd.DataFrame(panel_data)

        # 只留下需要处理的列
        cols = factor_list
        # 分组的列
        gp_col = 'industry_code'
        # 查询nan的列
        df_na = df[cols].isnull()
        # 根据分组计算中位数
        df_median = df.groupby(gp_col)[cols].median()
        # 依次处理每一列
        for col in cols:
            na_series = df_na[col]
            names = list(df.loc[na_series, gp_col])

            t = df_median.loc[names, col]
            t.index = df.loc[na_series, col].index
            t.fillna(t.mean(),inplace=True)


            # 相同的index进行赋值
            df.loc[na_series, col] = t

        if flag == 0:
            filled_df = df
            flag = 1
        else:
            filled_df = pd.concat([filled_df, df])


    #filled_df.to_csv("filled_data.csv", index=False, float_format="%.6f")
    return filled_df


factor_list=['BETA','RSTR','LNCAP','ETOP','DASTD','EGRO','BTOP','DTOA','STOM']
alpha_df=pd.read_csv("zz_data.csv")
df_1=median_fill(alpha_df,factor_list)


#中位数极值法去除极端值
def Median(alpha_df,factor_list):
    date_list = list(alpha_df["datetime"].drop_duplicates())
    grouped = alpha_df.groupby("datetime")

    corrected_df = pd.DataFrame()
    flag = 0

    for date in date_list:
        print(date,end=" ")
        try:
            panel_data = grouped.get_group(date)
        except:
            continue
        df = pd.DataFrame(panel_data)
        colomn_names = factor_list
        for colomn_name in colomn_names:
            arr = np.array(df[colomn_name])
            M = np.median(arr)
            MAD = 1.483 * np.median(np.abs(arr - M))
            arr[arr > M + 3 * MAD] = M + 3 * MAD
            arr[arr < M - 3 * MAD] = M - 3 * MAD
            df[colomn_name] = arr
        if flag == 0:
            flag = 1
            corrected_df = df
        else:
            corrected_df = pd.concat([corrected_df, df])

    #corrected_df.to_csv("median_mad.csv", index=False)
    return corrected_df

alpha_df=df_1
factor_list=['BETA','RSTR','LNCAP','ETOP','DASTD','EGRO','BTOP','DTOA','STOM']
df_2=Median(alpha_df,factor_list)


#Z_score法进行标准化
def Z_score(alpha_df,factor_list):
    date_list = list(alpha_df["datetime"].drop_duplicates())
    grouped = alpha_df.groupby("datetime")
    stddized_df = pd.DataFrame()
    flag = 0

    for date in date_list:
        print(date,end=" ")
        try:
            panel_data = grouped.get_group(date)
        except:
            continue
        df = pd.DataFrame(panel_data)

        colomn_names = factor_list
        for colomn_name in colomn_names:
            arr = np.array(df[colomn_name])
            weight = np.array(len(arr) * [1 / len(arr)])
            weighted_avarage = np.dot(weight.T, arr)
            arr = (arr - weighted_avarage) / arr.std()
            df[colomn_name] = arr
        if flag == 0:
            flag = 1
            stddized_df = df
        else:
            stddized_df = pd.concat([stddized_df, df])

    stddized_df.to_csv("equal_weight ZScore.csv", index=False)


factor_list=['BETA','RSTR','LNCAP','ETOP','DASTD','EGRO','BTOP','DTOA','STOM']
alpha_df=df_2
Z_score(alpha_df,factor_list)

In [ ]:
#添加上股票的日频收益率数据
zz_data=pd.read_csv("equal_weight ZScore.csv")
l=list(set(zz_data["code"][zz_data["industry_code"]==0]))
zz_data.set_index("code",inplace=True)
zz_data.drop(l,inplace=True)
zz_data.reset_index(inplace=True)

change_data_1=pd.read_csv("change.csv",dtype={"stock_code":str})
change_data_2=pd.read_csv("change_2.csv",dtype={"stock_code":str})
change_data=pd.concat([change_data_1,change_data_2])

change_data.rename(columns={"stock_code":"code","trade_date":"datetime"},inplace=True)
change_data["datetime"]=change_data["datetime"].apply(lambda x:x[0:10])
change_data["code"]=change_data["code"].apply(lambda x:x+".SH" if x[0]=="6" else x+".SZ")
zz_data_1=pd.merge(zz_data,change_data,"left",on=["code","datetime"])

zz_data_1["change_rate"].fillna(0,inplace=True)

In [ ]:
#根据股票日频收益率计算出股票21天收益率以及年化21天平均收益
def fun(df):
    df=pd.DataFrame(df)
    df.reset_index(inplace=True)
    
    df["ln_change"]=np.log(1+df["change_rate"]/100)
    df["21d_ret"]=(np.exp(pd.rolling_sum(df["ln_change"],21))-1)*100
    df["avg_ret"]=pd.rolling_mean(df["21d_ret"][::-1],200)[::-1].shift(-20)
    
    index=df["index"]
    new_df["avg_ret"][index]=df["avg_ret"]
    new_df["21d_ret"][index]=df["21d_ret"]
    print("finish",end=" ")
    return 0

new_df=pd.DataFrame(zz_data_1)
new_df["21d_ret"]=0
new_df["avg_ret"]=0
zz_data_1.groupby("code").apply(fun)

new_df.to_csv("equal_weight ZScore.csv",index=False)